<a href="https://colab.research.google.com/github/AshwinPradeep01/EmotzyCare-The-Multimodal-Emotion-Recognizer/blob/main/EmotionDetectionUsingBert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Sun Dec  3 05:11:10 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   64C    P8    11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Datasets

Mounted at /content/drive
/content/drive/MyDrive/Datasets


In [3]:
import tensorflow as tf
!pip install transformers
from transformers import BertModel, BertTokenizer
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

In [4]:
#Setting up the GPU
from torch import cuda
device =  'cuda' if cuda.is_available() else 'cpu'
device

'cuda'

In [5]:
emotions = pd.read_csv('/content/drive/MyDrive/Datasets/iemocapTrans.csv')
emotions.head()

,_id,activation,dominance,emotion,end_time,start_time,titre,to_translate,translated,valence
0,625682441da7a5c1eaef3689,2.5,3.5,sad,6.0541,3.9987,Ses02M_impro02_F000,I don't want you to go.,Je ne veux pas que tu partes.,2.5
1,625682441da7a5c1eaef368a,3.0,4.0,sad,15.1000,7.0366,Ses02M_impro02_M000,"I know, I know. I don't want to go either bab...",Je sais je sais. Je ne veux pas y aller non pl...,2.0
2,625682441da7a5c1eaef368b,2.5,4.5,sad,23.3599,15.5524,Ses02M_impro02_F001,I'm going to miss you too; I don't know what ...,Tu vas me manquer aussi; Je ne sais pas ce que...,1.5
3,625682441da7a5c1eaef368c,2.5,4.0,sad,26.4151,23.5790,Ses02M_impro02_F002,I don't want to be a single mom.,Je ne veux pas être une mère célibataire.,1.5
4,625682441da7a5c1eaef368d,3.0,3.5,sad,31.4253,26.7598,Ses02M_impro02_M001,You won't be. I'll be back; I'll be back befo...,Vous ne le serez pas. Je reviendrai; Je serai ...,3.5


In [6]:
emotions = emotions.loc[:,['emotion','to_translate']]
emotions.head()


,emotion,to_translate
0,sad,I don't want you to go.
1,sad,"I know, I know. I don't want to go either bab..."
2,sad,I'm going to miss you too; I don't know what ...
3,sad,I don't want to be a single mom.
4,sad,You won't be. I'll be back; I'll be back befo...


In [7]:
emotions.rename(columns={'to_translate':'text'},inplace =True)
emotions.head()

,emotion,text
0,sad,I don't want you to go.
1,sad,"I know, I know. I don't want to go either bab..."
2,sad,I'm going to miss you too; I don't know what ...
3,sad,I don't want to be a single mom.
4,sad,You won't be. I'll be back; I'll be back befo...


In [8]:
LABEL_COLUMNS = ['ang', 'exc', 'fru', 'hap', 'neu', 'sad', 'sur']

In [9]:
print(emotions['emotion'].unique())

['sad' 'fru' 'neu' 'hap' 'exc' 'sur' 'ang' 'fea' 'oth' 'dis']


In [10]:
print("The Frequency of the labels: \n", pd.value_counts(emotions['emotion']))

The Frequency of the labels: 
 fru    2917
exc    1976
neu    1726
ang    1269
sad    1250
hap     656
sur     110
fea     107
oth      26
dis       2
Name: emotion, dtype: int64


In [11]:
#Emotions dictionary
emotion_dictionary = {
    'happy' :['exc','hap', 'sur'],
    'sad' : ['sad'],
    'anger' : ['ang'],
    'frustrated': ['fru'],
    'neutral' : ['neu']
}

In [12]:
print(emotions.dtypes)

emotion    object
text       object
dtype: object


In [13]:
# emotions['emotion'].replace('fru','ang',inplace =True)
emotions['emotion'].replace(['exc','sur'],'hap', inplace =True)
emotions['emotion'].replace('dis','sad', inplace =True)


In [14]:
index_def =emotions[(emotions['emotion']=='oth')|(emotions['emotion']=='fea')].index
emotions = emotions.drop(index_def,axis =0)
emotions['emotion'].unique()

array(['sad', 'fru', 'neu', 'hap', 'ang'], dtype=object)

In [15]:
df = emotions
df.head()

,emotion,text
0,sad,I don't want you to go.
1,sad,"I know, I know. I don't want to go either bab..."
2,sad,I'm going to miss you too; I don't know what ...
3,sad,I don't want to be a single mom.
4,sad,You won't be. I'll be back; I'll be back befo...


In [16]:
# from sklearn.preprocessing import LabelEncoder
# le = LabelEncoder()
# df['emotion'] = le.fit_transform(df['emotion'])
# le.classes_
# # df.head()
# df.emotion

In [17]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [18]:
# Defining some key variables that will be used later on in the training
MAX_LEN = 512
TRAIN_BATCH_SIZE = 8
VALID_BATCH_SIZE = 8
EPOCHS = 6
LEARNING_RATE = 5e-05

In [19]:
df.emotion.value_counts()

fru    2917
hap    2742
neu    1726
ang    1269
sad    1252
Name: emotion, dtype: int64

In [20]:
labels = sorted(df.emotion.unique())
df.emotion.unique()

array(['sad', 'fru', 'neu', 'hap', 'ang'], dtype=object)

In [21]:
label_dict= {}
for index, label in enumerate(labels):
  label_dict[label]=index

In [22]:
label_dict

{'ang': 0, 'fru': 1, 'hap': 2, 'neu': 3, 'sad': 4}

In [23]:
df.emotion = df['emotion'].map(label_dict)

In [24]:
df.emotion

0        4
1        4
2        4
3        4
4        4
        ..
10034    0
10035    1
10036    2
10037    2
10038    2
Name: emotion, Length: 9906, dtype: int64

In [25]:
from sklearn.model_selection import train_test_split

In [26]:
X_train, X_val, y_train, y_val = train_test_split(df.index.values,
                                                  df.emotion.values,
                                                  test_size=0.20,
                                                  random_state=42,
                                                  stratify=df.emotion.values)

In [27]:
df['data_type'] = ['not_set']*df.shape[0]

In [28]:
df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'

In [29]:
df.groupby(['emotion', 'data_type']).count()

text
emotion data_type      
0       train      1015
        val         254
1       train      2333
        val         584
2       train      2193
        val         549
3       train      1381
        val         345
4       train      1002
        val         250

In [30]:
df.text.isnull().sum()

0

In [31]:
df.dropna(inplace =True)

## **Using Bert Tokenizer**

In [32]:
import torch
from transformers import BertTokenizer
from torch.utils.data import TensorDataset

In [33]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased',do_lower_case=True)

In [34]:
# Save the tokenizer to a file
tokenizer.save_pretrained('/content/sample_data')
loaded_tokenizer = BertTokenizer.from_pretrained('/content/sample_data')

In [35]:
text = "This is an example sentence."
tokens = loaded_tokenizer.tokenize(text)
ids = loaded_tokenizer.convert_tokens_to_ids(tokens)
ids

[2023, 2003, 2019, 2742, 6251, 1012]

In [36]:
EPOCHS = 10
LEARNING_RATE = 1e-05

In [37]:
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type=='train'].text.values,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=256,
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type=='val'].text.values,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=256,
    return_tensors='pt'
)

input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type=='train'].emotion.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type=='val'].emotion.values)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2614: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [38]:
dataset_train = TensorDataset(input_ids_train,
                              attention_masks_train,
                              labels_train)

dataset_val = TensorDataset(input_ids_val,
                            attention_masks_val,
                           labels_val)

In [39]:
len(dataset_train)

7924

In [40]:
dataset_val.tensors

(tensor([[  101,  1037,  2210,  ...,     0,     0,     0],
         [  101,  1045,  3246,  ...,     0,     0,     0],
         [  101,  2017, 10657,  ...,     0,     0,     0],
         ...,
         [  101,  2092,  1045,  ...,     0,     0,     0],
         [  101,  3492,  2172,  ...,     0,     0,     0],
         [  101,  2002,  2074,  ...,     0,     0,     0]]),
 tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]]),
 tensor([1, 4, 1,  ..., 4, 4, 4]))

In [41]:
from transformers import BertForSequenceClassification

In [42]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased',num_labels = len(label_dict),
                                                      output_attentions = False,output_hidden_states = False)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [43]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

In [44]:
batch_size = 4

dataloader_train = DataLoader(
    dataset_train,
    sampler=RandomSampler(dataset_train),
    batch_size=batch_size
)

dataloader_val = DataLoader(
    dataset_val,
    sampler=RandomSampler(dataset_val),
    batch_size=32
)

## **Initializing the optimizer (ADAMW)**

In [45]:
from transformers import AdamW, get_linear_schedule_with_warmup

optimizer = torch.optim.AdamW(params = model.parameters(), lr = LEARNING_RATE,eps = 1e-8)

In [46]:
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps = 0,
                                            num_training_steps=len(dataloader_train)*EPOCHS)

In [47]:
from sklearn.metrics import f1_score

def score(preds, labels):
  preds_flat = np.argmax(preds, axis=1).flatten()
  labels_flat = labels.flatten()
  return f1_score(labels_flat, preds_flat, average = 'weighted')

##**Accuracy per class**

In [48]:
def accuracy(preds, labels):
  label_dict_inverse = {v: k for k, v in label_dict.items()}
  accu =0
  preds_flat = np.argmax(preds, axis=1).flatten()
  labels_flat = labels.flatten()

  for label in np.unique(labels_flat):
    y_preds = preds_flat[labels_flat==label]
    y_true = labels_flat[labels_flat==label]
    print(f'Class: {label_dict_inverse[label]}')
    accu = (len(y_preds[y_preds==label])/len(y_true))*100
    print(f'Accuracy:{accu}%\n')

## Training module

In [49]:
import random

RANDOM_SEED = 42
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
torch.cuda.manual_seed_all(RANDOM_SEED)

In [50]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [51]:
def evaluate(dataloader_val):

    model.eval()

    loss_val_total = 0
    predictions, true_vals = [], []

    for batch in tqdm(dataloader_val):

        batch = tuple(b.to(device) for b in batch)

        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():
            outputs = model(**inputs)

        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)

    loss_val_avg = loss_val_total/len(dataloader_val)

    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)

    return loss_val_avg, predictions, true_vals

In [52]:
from tqdm.notebook import tqdm

In [53]:
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

## **Training Loop**

Here the trainings epochs along with their training loss and validation loss have been illustrated.

In [54]:
for epoch in tqdm(range(1, EPOCHS+1)):
    model.train()
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train,
                        desc=f'Epoch {epoch}',
                        leave=False,
                        disable=False)

    for batch in progress_bar:
        model.zero_grad()
        batch = tuple(b.to(device) for b in batch)
        inputs = {
            'input_ids': batch[0],
            'attention_mask': batch[1],
            'labels': batch[2]
        }

        outputs = model(**inputs)
        loss = outputs[0]
        loss_train_total +=loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()

        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})

    #torch.save(model.state_dict(), f'Models/BERT_ft_Epoch{epoch}.model')

    tqdm.write('\nEpoch {epoch}')

    loss_train_avg = loss_train_total/len(dataloader_train)
    tqdm.write(f'Training loss: {loss_train_avg}')

    val_loss, predictions, true_vals = evaluate(dataloader_val)
    val_f1 = score(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (weighted): {val_f1}')

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/1981 [00:00<?, ?it/s]


Epoch {epoch}
Training loss: 1.3029521614659139


  0%|          | 0/62 [00:00<?, ?it/s]

Validation loss: 1.1070043117769304
F1 Score (weighted): 0.563232265742813


...

Validation loss: 2.0892939932884707
F1 Score (weighted): 0.5928672927867902


Epoch 10:   0%|          | 0/1981 [00:00<?, ?it/s]


Epoch {epoch}
Training loss: 0.38915229330708734


  0%|          | 0/62 [00:00<?, ?it/s]

Validation loss: 2.134391409735526
F1 Score (weighted): 0.5866143547270113


In [2]:
torch.save(model,'/content/drive/MyDrive/Datasets/em\otzy_model.pth')
tokenizer.save_vocabulary('/content/drive/MyDrive/Datasets/emotzy_vocab_model.pth')

NameError: ignored

In [56]:
accuracy(predictions, true_vals)

Class: ang
Accuracy:44.09448818897638%

Class: fru
Accuracy:54.62328767123288%

Class: hap
Accuracy:74.68123861566485%

Class: neu
Accuracy:52.46376811594203%

Class: sad
Accuracy:57.599999999999994%



In [3]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Datasets

Mounted at /content/drive
/content/drive/MyDrive/Datasets


In [4]:
import torch
from transformers import BertTokenizer
from torch.utils.data import TensorDataset

In [5]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased',do_lower_case=True)

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [6]:
bertmodel = torch.load('/content/drive/MyDrive/Datasets/emotzy_model.pth',map_location=torch.device('cpu'))

In [7]:
review_text = "This makes me completely drained and emotionally tiresome"

review_text2 = "I m on cloud nine feeling so delighted"

def get_emotion(review_text):
    encoded_review = tokenizer.encode_plus(review_text, max_length=512,
                                          add_special_tokens=True, return_token_type_ids = False,
                                          pad_to_max_length=True,
                                          return_attention_mask=True,
                                          truncation=True,
                                          return_tensors='pt')
    input_ids = encoded_review['input_ids']
    attention_mask = encoded_review['attention_mask']
    output = bertmodel(input_ids,attention_mask)
    prediction = int(torch.argmax(output.logits))
    pred_dict ={
        0:"angry",
        1:"frustrated",
        2:"happy",
        3:"neutral",
        4:"sad"
    }
    prediction = pred_dict[prediction]
    print(f"Review text:{review_text}")
    print(f"Sentiment:{prediction}")

get_emotion(review_text2)
get_emotion(review_text)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2614: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Review text:I m on cloud nine feeling so delighted
Sentiment:happy


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2614: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Review text:This makes me completely drained and emotionally tiresome
Sentiment:frustrated
